In [16]:
import sys
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [18]:
import pandas as pd
import numpy as np
import os

import joblib

import tensorflow.keras
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, MaxPooling2D, Conv2D, LSTM, GRU, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [19]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NonPromoterSequence.txt', sep = '>', )
df.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df.reset_index(inplace = True)
df.drop(['EP 1 (+) mt:CoI_1; range -400 to -100.', 'index'], axis = 1, inplace=True) #data cleaning after error found
df.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df['label'] = 0
display(df)
display(df.shape)

,sequence,label
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
2,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0
3,TATGTATATAGAGATAGGCGTTGCCAATAACTTTTGCGTTTTTTGC...,0
4,AGAAATAATAGCTAGAGCAAAAAACAGCTTAGAACGGCTGATGCTC...,0
...,...,...
11295,TGGTAAAAAATTGTACACCTAACTAGTGCCTTCATGTATACCACCA...,0
11296,AGTGCAACTGGAGCCGTGCCGTGACCCACAGAGATCGCCCACTCGA...,0
11297,GCATGGATTTCATATTATCTTAATCGACTTGCTTTTATAAAATAGG...,0
11298,GTGACCAGGTTTTGCTCTAATGCGAAGTACGGATTGGGTAGAGATA...,0


(11300, 2)

In [20]:
df2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/PromoterSequence.txt', sep = '>', )
df2.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df2.reset_index(inplace = True)
df2.drop(['EP 1 (+) mt:CoI_1; range -100 to 200.', 'index'], axis = 1, inplace=True)
df2.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df2['label'] = 1

display(df2)
display(df2.shape)


,sequence,label
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1
...,...,...
11295,CGACAAAGTTTGATCCATGTGCATTCTTGGCGCCTTATCGATAGCT...,1
11296,CATATCTACATCTCGCTTGCTCCTTCCCTTTCGCTGCGTGTGTGTG...,1
11297,ATACCGCGGAAGCGCAAAAGTACCAGAATTTCCCTGGTATCGCGCT...,1
11298,ATTATTCCGAATTCTTTTATCAGATTTAAATATGGGAAACACTTTA...,1


(11300, 2)

In [21]:
df = pd.concat([df, df2], axis = 0 )
df.shape


(22600, 2)

In [22]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

,sequence,label
1822,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTC...,0


In [23]:
df.drop([1822], inplace = True)

In [24]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

In [25]:
sequence = list(df.loc[:, 'sequence'])
encoded_list = []


In [26]:
##have to change into see window k=3,5,7
#최댓값인 7을 기준으로 encoding vector 의 크기를 정하면 크기는 4^7 = 16384
#입력으로 사용하기에는 너무 sparse 하고 크기가 크기 때문에
#autoencoder 같은 것을 이용해서 dense vector로 만드는 것도 필요할 것 같음

#우선은 window k = 3 을 기준으로 구현
#input vector 가 4 -> 64

# def encode_seq(s):
#   Encode = {'A': 0,'T': 1,'C': 2,'G': 3}
#   ret = []
#   for i in range(len(s) - 2):
#     encode_vec = [0 for j in range(64)]
#     encode_index = 0
#     encode_index += Encode[s[i]]
#     encode_index += Encode[s[i+1]]*4
#     encode_index += Encode[s[i+2]]*16
#     encode_vec[encode_index] = 1
#     # print(encode_vec)
#     # print(ret)
#     ret.append(encode_vec)
#   return ret

#64 = 2^6 이므로 dim=7 인 벡터로 임베딩
#[0,0,0,0,0,0,1] 부터 [1,0,0,0,0,0,0]
#이렇게 할 경우 one-hot vector로 구현하는 것이 아니기 때문에 self-supervised prediction 같은 방식으로
#Transformer 나 bidirectional RNN layer 로 
#DNA sequence에 대한 이해를 얻게 만드는 것은 불가능하지만
#현재 구현 하에서의 CNN-RNN-dense 구조로 바로 label을 예측하는 것은 학습시킬 수 있다.

#다시 생각해봐야되나 싶음. RNN (GRU) 내부에 softmax function이 있을 경우에는 one-hot encoding이 아닌 경우는 부적절함
#실제 학습 결과 15 epoch 에서 accuracy가 70% 밖에 나오지 않음, 부적절한 인코딩인듯

def encode_seq(s):
  Encode = {'A': 0,'T': 1,'C': 2,'G': 3}
  ret = []
  for i in range(len(s) - 2):
    encode_vec = [0 for j in range(7)]
    encode_index = 0
    encode_index += Encode[s[i]]
    encode_index += Encode[s[i+1]]*4
    encode_index += Encode[s[i+2]]*16
    for j in range(7):
      if encode_index % 2 == 1:
        encode_vec[j] = 1
      encode_index = encode_index / 2
    ret.append(encode_vec)
  return ret

for i in sequence:
  x = encode_seq(i)
  encoded_list.append(x)


# def encode_seq(s):
#     Encode = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1]}
#     return [Encode[x] for x in s]

# for i in sequence:
#     x = encode_seq(i)
#     encoded_list.append(x)

X = np.array(encoded_list)
X.shape



(22598, 299, 7)

In [27]:
y = df['label']
y.shape
X.shape

(22598, 299, 7)

In [28]:
X_train, X_not_train, y_train, y_not_train = train_test_split(X, y, random_state = 42, stratify = y)
y_train = utils.to_categorical(y_train)

X_val, X_test, y_val, y_test = train_test_split(X_not_train, y_not_train, random_state = 42, stratify = y_not_train, test_size=0.2)

y_val = utils.to_categorical(y_val)
y_test = utils.to_categorical(y_test)

X_train = tf.cast(X_train, dtype='float32')
X_val = tf.cast(X_val, dtype='float32')
X_test = tf.cast(X_test, dtype='float32')

params = {
    'first_node': [128, 64],
    'second_node': [32, 64],
    'alpha': [0.001, 0.01],
    'first_filter': [9, 16, 32], 
    'dropout': [0.1, 0.2, 0.5]
}
with tensorflow.device('/device:GPU:0'):
  gru_model = Sequential()

  #원래 cnn의 구조는
  #(301, 4) -> (208, 27) -> (206, 27) -> (205, 14)?
  #단순히 (301,64) input을 GRU에 넣으니 램이 초과됨
  #차원을 줄여서 넣을 필요가 있음
  #일단 Conv1D로 64 -> 1 로 줄이는 걸 시도해봄
  #(301, 64) -> (208, 27) -> (206, 27) -> (205, 14)?
  #여전히 램 초과
  #더 나은 sequence encoding 이 필요할듯
  
  #one-hot vector encoding 을 포기했음

  #단순하게 301*4 input을 input으로 사용하고 cnn layer를 사용하지 않은 경우에도
  #input length가 너무 길기 때문에 학습 시간이 느리다는 단점은 있었지ㄴ
  #50 epoch 정도 이후에 accuracy 가 85% 정도가 나왔고 이후 85~90% 사이를 유지했음
  #즉 CNN layer는 input size를 줄여 RNN의 느린 학습 속도를 해결해주는 방법이지
  #그 자체로 성능을 증가시켜주지는 않는 것 같음 (CNN+RNN 은 88% 정도였기 때문)

  #또 only rnn 은 학습 속도가 느려서 colab 제한 시간 내에 115 epoch가 학습이 불가능, 90이 3시간 이상 걸렸고, 중간에 끊겼음
  #따라서 window+RNN 은 40 epoch 만 진행할 것

  #아마 transformer를 도입하지 않으면 성능 증가는 힘들 것 같음.
  #RNN <= CNN+RNN <= Transforemr 일 것 같음

  
  # gru_model.add(Conv1D(filters = 27, kernel_size = (4), activation = 'relu', input_shape = (301, 64)))
  # gru_model.add(MaxPooling1D(pool_size= (3)))
  # gru_model.add(Dropout(0.2))

  # gru_model.add(Conv1D(filters = 14, kernel_size = (2), activation = 'relu', padding = 'same'))

  # #cnn_model.add(MaxPooling1D(pool_size= (1)))
  # #cnn_model.add(Dropout(0.2))


  gru_model.add(Bidirectional(GRU(128, activation = 'relu')))
  gru_model.add(Dropout(0.2))
  gru_model.add(Dense(128, activation = 'relu'))
  gru_model.add(Dense(64, activation = 'relu'))
  gru_model.add(Dense(64, activation = 'relu'))
  gru_model.add(Dense(16, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
  gru_model.add(Dense(2, activation = 'sigmoid'))

  gru_model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  early_stop = tensorflow.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta = 0.0005, patience=8, 
                                            restore_best_weights=True )
  history = gru_model.fit(X_train, y_train, batch_size = 128, validation_data=(X_val, y_val), 
                          epochs=40)

Epoch 1/40
133/133 [==============================] - 193s 1s/step - loss: 0.8265 - accuracy: 0.5353 - val_loss: 0.7416 - val_accuracy: 0.5653
Epoch 2/40
133/133 [==============================] - 158s 1s/step - loss: 0.7344 - accuracy: 0.5582 - val_loss: 0.7029 - val_accuracy: 0.5471
Epoch 3/40
133/133 [==============================] - 159s 1s/step - loss: 0.6745 - accuracy: 0.6018 - val_loss: 0.6140 - val_accuracy: 0.6872
Epoch 4/40
133/133 [==============================] - 158s 1s/step - loss: 0.6104 - accuracy: 0.6872 - val_loss: 0.5794 - val_accuracy: 0.7038
Epoch 5/40
133/133 [==============================] - 156s 1s/step - loss: 0.5923 - accuracy: 0.6956 - val_loss: 0.5897 - val_accuracy: 0.7073
Epoch 6/40
133/133 [==============================] - 155s 1s/step - loss: 0.5849 - accuracy: 0.7011 - val_loss: 0.5883 - val_accuracy: 0.6942
Epoch 7/40
133/133 [==============================] - 157s 1s/step - loss: 0.5812 - accuracy: 0.7034 - val_loss: 0.5700 - val_accuracy: 0.7131

In [29]:
pred = gru_model.predict

In [30]:
preds = gru_model.predict(X_test)

print(X_test.shape)
print(preds)
print(preds.shape)
for i in range(preds.shape[0]):
  if preds[i][0] >= preds[i][1]:
    preds[i][0] = 1
    preds[i][1] = 0
  else:
    preds[i][0] = 0
    preds[i][1] = 1
print(preds)
print(y_test)
print(y_test.shape)

np.savetxt("test_predictions_window_3.csv", preds, delimiter=",")

36/36 [==============================] - 4s 90ms/step
(1130, 299, 7)
[[0.5636711  0.43827116]
 [0.4896648  0.5179167 ]
 [0.5958217  0.40152735]
 ...
 [0.75125706 0.24686575]
 [0.61430603 0.38303664]
 [0.6723946  0.32524332]]
(1130, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
(1130, 2)
